In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('Metro-Subsidence.xlsx')

In [3]:
first_four_columns = df.iloc[:, :5]
print(first_four_columns)

     distance_from_start          X_UTM         Y_UTM  val_20150220  \
0                      0  719349.660300  4.024909e+06             0   
1                    101  719255.141857  4.024944e+06             0   
2                    202  719160.623414  4.024980e+06             0   
3                    303  719066.104972  4.025015e+06             0   
4                    404  718971.586536  4.025051e+06             0   
..                   ...            ...           ...           ...   
436                44036  691022.006470  4.055924e+06             0   
437                44137  690937.161678  4.055979e+06             0   
438                44238  690852.316886  4.056034e+06             0   
439                44339  690767.472094  4.056089e+06             0   
440                44440  690682.627303  4.056143e+06             0   

     val_20150304  
0       -2.195996  
1       -2.340330  
2       -2.376158  
3       -2.138846  
4       -1.795333  
..            ...  
436    

In [13]:
!ls *.tif

LULC_2017_10m.tif  LULC_2019_10m.tif  LULC_2022_10m.tif  LULC_2024_10m.tif
LULC_2018_10m.tif  LULC_2021_10m.tif  LULC_2023_10m.tif


In [15]:
import rasterio
from pathlib import Path

raster_path = Path.cwd() / 'LULC_2017_10m.tif'
with rasterio.open(raster_path) as src:
    print(f"Raster CRS: {src.crs}")

Raster CRS: GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]


In [4]:
import geopandas as gpd
import rasterio
import pandas as pd
from pathlib import Path
from rasterio.crs import CRS

# Load your DataFrame (assuming it's already loaded as `df`)
# If not, you can load it, e.g., df = pd.read_csv('your_file.csv')

# Convert the DataFrame to a GeoDataFrame with UTM Zone 41N
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['X_UTM'], df['Y_UTM']),
    crs="EPSG:32641"  # UTM Zone 41N
)

# Reproject points to WGS84 (EPSG:4326) to match raster CRS
gdf_wgs84 = gdf.to_crs("EPSG:4326")

# Directory containing the raster files (same as script)
raster_dir = Path.cwd()

# List of raster files
raster_files = [
    'LULC_2017_10m.tif', 'LULC_2018_10m.tif', 'LULC_2019_10m.tif',
    'LULC_2021_10m.tif', 'LULC_2022_10m.tif', 'LULC_2023_10m.tif', 'LULC_2024_10m.tif'
]

# Function to extract raster values at point locations
def extract_raster_values(raster_path, points_gdf):
    with rasterio.open(raster_path) as src:
        # Normalize and check if raster CRS is equivalent to WGS84
        wgs84_crs = CRS.from_epsg(4326)
        if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:
            raise ValueError(f"Raster {raster_path} is not in WGS84 (expected EPSG:4326, got {src.crs})")
        
        # Get coordinates of points
        coords = [(x, y) for x, y in zip(points_gdf.geometry.x, points_gdf.geometry.y)]
        
        # Sample the raster at point locations
        values = [val[0] for val in src.sample(coords)]
    
    return values

# Loop through each raster and add values to the GeoDataFrame
for raster_file in raster_files:
    raster_path = raster_dir / raster_file
    if not raster_path.exists():
        print(f"Warning: {raster_path} not found, skipping...")
        continue
    
    year = raster_file.split('_')[1].split('_')[0]  # Extract year (e.g., '2017')
    column_name = f'LULC_{year}'
    
    # Extract values and add as a new column
    gdf[column_name] = extract_raster_values(raster_path, gdf_wgs84)

# Convert back to a regular DataFrame (drop geometry)
result_df = pd.DataFrame(gdf.drop(columns='geometry'))

# Save or print the resulting DataFrame
print(result_df)
# Optionally, save to CSV
result_df.to_csv('output_with_lulc.csv', index=False)

/tmp/ipykernel_115593/629527331.py:34: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:
/tmp/ipykernel_115593/629527331.py:34: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:


     distance_from_start          X_UTM         Y_UTM  val_20150220  \
0                      0  719349.660300  4.024909e+06             0   
1                    101  719255.141857  4.024944e+06             0   
2                    202  719160.623414  4.024980e+06             0   
3                    303  719066.104972  4.025015e+06             0   
4                    404  718971.586536  4.025051e+06             0   
..                   ...            ...           ...           ...   
436                44036  691022.006470  4.055924e+06             0   
437                44137  690937.161678  4.055979e+06             0   
438                44238  690852.316886  4.056034e+06             0   
439                44339  690767.472094  4.056089e+06             0   
440                44440  690682.627303  4.056143e+06             0   

     val_20150304  val_20150328  val_20150421  val_20150515  val_20150702  \
0       -2.195996     -0.942629      1.503353      0.788563      7.239

/tmp/ipykernel_115593/629527331.py:34: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:
/tmp/ipykernel_115593/629527331.py:34: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:
/tmp/ipykernel_115593/629527331.py:34: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:
/tmp/ipykernel_115593/629527331.py:34: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:
/tmp/ipykernel_115593/629527331.py:34: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or not sr

In [5]:
import rasterio
from pathlib import Path

# Assuming `gdf_wgs84` is your reprojected GeoDataFrame from the script
# Print point bounds (in WGS84)
print("Point bounds (min_lon, min_lat, max_lon, max_lat):", gdf_wgs84.total_bounds)

# Check bounds for each raster
raster_files = [
    'LULC_2017_10m.tif', 'LULC_2018_10m.tif', 'LULC_2019_10m.tif',
    'LULC_2021_10m.tif', 'LULC_2022_10m.tif', 'LULC_2023_10m.tif', 'LULC_2024_10m.tif'
]
raster_dir = Path.cwd()

for raster_file in raster_files:
    raster_path = raster_dir / raster_file
    if raster_path.exists():
        with rasterio.open(raster_path) as src:
            print(f"{raster_file} bounds (min_lon, min_lat, max_lon, max_lat):", src.bounds)

Point bounds (min_lon, min_lat, max_lon, max_lat): [65.13269399 36.34427611 65.4442135  36.63177655]
LULC_2017_10m.tif bounds (min_lon, min_lat, max_lon, max_lat): BoundingBox(left=59.09073023174007, bottom=36.310083447167884, right=59.55668636961286, top=36.65288055958789)
LULC_2018_10m.tif bounds (min_lon, min_lat, max_lon, max_lat): BoundingBox(left=59.09073023174007, bottom=36.310083447167884, right=59.55668636961286, top=36.65288055958789)
LULC_2019_10m.tif bounds (min_lon, min_lat, max_lon, max_lat): BoundingBox(left=59.09073023174007, bottom=36.310083447167884, right=59.55668636961286, top=36.65288055958789)
LULC_2021_10m.tif bounds (min_lon, min_lat, max_lon, max_lat): BoundingBox(left=59.09073023174007, bottom=36.310083447167884, right=59.55668636961286, top=36.65288055958789)
LULC_2022_10m.tif bounds (min_lon, min_lat, max_lon, max_lat): BoundingBox(left=59.09073023174007, bottom=36.310083447167884, right=59.55668636961286, top=36.65288055958789)
LULC_2023_10m.tif bounds (min

In [6]:
import rasterio
import numpy as np

for raster_file in raster_files:
    raster_path = raster_dir / raster_file
    if raster_path.exists():
        with rasterio.open(raster_path) as src:
            print(f"{raster_file}:")
            print(f"  Nodata value: {src.nodata}")
            print(f"  Number of bands: {src.count}")
            # Read the first band and check unique values
            data = src.read(1)  # Read first band
            unique_values = np.unique(data)
            print(f"  Unique values in first band: {unique_values}")

LULC_2017_10m.tif:
  Nodata value: None
  Number of bands: 1
  Unique values in first band: [0 1 2 3 4 5 6 9]
LULC_2018_10m.tif:
  Nodata value: None
  Number of bands: 1
  Unique values in first band: [0 1 2 4 5 6 9]
LULC_2019_10m.tif:
  Nodata value: None
  Number of bands: 1
  Unique values in first band: [0 1 2 4 5 6 9]
LULC_2021_10m.tif:
  Nodata value: None
  Number of bands: 1
  Unique values in first band: [0 1 2 3 4 5 6 9]
LULC_2022_10m.tif:
  Nodata value: None
  Number of bands: 1
  Unique values in first band: [0 1 2 3 4 5 6 7 9]
LULC_2023_10m.tif:
  Nodata value: None
  Number of bands: 1
  Unique values in first band: [0 1 2 3 4 5 6 9]
LULC_2024_10m.tif:
  Nodata value: None
  Number of bands: 1
  Unique values in first band: [0 1 2 3 4 5 6 9]


In [7]:
# Assuming `gdf` and `gdf_wgs84` are from your script
print("First 5 points in UTM Zone 41N (X_UTM, Y_UTM):")
print(gdf[['X_UTM', 'Y_UTM']].head())
print("\nFirst 5 points in WGS84 (lon, lat):")
print(gdf_wgs84[['geometry']].head().apply(lambda x: (x.geometry.x, x.geometry.y)))


First 5 points in UTM Zone 41N (X_UTM, Y_UTM):
           X_UTM         Y_UTM
0  719349.660300  4.024909e+06
1  719255.141857  4.024944e+06
2  719160.623414  4.024980e+06
3  719066.104972  4.025015e+06
4  718971.586536  4.025051e+06

First 5 points in WGS84 (lon, lat):
                                            geometry
0  0    65.444214
1    65.443171
2    65.442129
3...
1  0    36.344276
1    36.344618
2    36.344960
3...


In [8]:
import rasterio

raster_path = raster_dir / 'LULC_2017_10m.tif'
coords = [(x, y) for x, y in zip(gdf_wgs84.geometry.x[:5], gdf_wgs84.geometry.y[:5])]
with rasterio.open(raster_path) as src:
    values = [val[0] for val in src.sample(coords)]
print("Sampled values for first 5 points:", values)

Sampled values for first 5 points: [np.uint8(0), np.uint8(0), np.uint8(0), np.uint8(0), np.uint8(0)]


In [9]:
import rasterio
from pathlib import Path

# Assuming `gdf_wgs84` is your reprojected GeoDataFrame (in WGS84)
print("Point bounds (min_lon, min_lat, max_lon, max_lat):", gdf_wgs84.total_bounds)

# Check bounds for each raster
raster_files = [
    'LULC_2017_10m.tif', 'LULC_2018_10m.tif', 'LULC_2019_10m.tif',
    'LULC_2021_10m.tif', 'LULC_2022_10m.tif', 'LULC_2023_10m.tif', 'LULC_2024_10m.tif'
]
raster_dir = Path.cwd()

for raster_file in raster_files:
    raster_path = raster_dir / raster_file
    if raster_path.exists():
        with rasterio.open(raster_path) as src:
            print(f"{raster_file} bounds (min_lon, min_lat, max_lon, max_lat):", src.bounds)

Point bounds (min_lon, min_lat, max_lon, max_lat): [65.13269399 36.34427611 65.4442135  36.63177655]
LULC_2017_10m.tif bounds (min_lon, min_lat, max_lon, max_lat): BoundingBox(left=59.09073023174007, bottom=36.310083447167884, right=59.55668636961286, top=36.65288055958789)
LULC_2018_10m.tif bounds (min_lon, min_lat, max_lon, max_lat): BoundingBox(left=59.09073023174007, bottom=36.310083447167884, right=59.55668636961286, top=36.65288055958789)
LULC_2019_10m.tif bounds (min_lon, min_lat, max_lon, max_lat): BoundingBox(left=59.09073023174007, bottom=36.310083447167884, right=59.55668636961286, top=36.65288055958789)
LULC_2021_10m.tif bounds (min_lon, min_lat, max_lon, max_lat): BoundingBox(left=59.09073023174007, bottom=36.310083447167884, right=59.55668636961286, top=36.65288055958789)
LULC_2022_10m.tif bounds (min_lon, min_lat, max_lon, max_lat): BoundingBox(left=59.09073023174007, bottom=36.310083447167884, right=59.55668636961286, top=36.65288055958789)
LULC_2023_10m.tif bounds (min

In [11]:
import geopandas as gpd
import rasterio
import pandas as pd
from pathlib import Path
from rasterio.crs import CRS
import numpy as np
from rasterio.warp import transform

# Load your DataFrame (assuming it's already loaded as `df`)
# If not, e.g., df = pd.read_csv('your_file.csv')

# Convert the DataFrame to a GeoDataFrame with UTM Zone 41N
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['X_UTM'], df['Y_UTM']),
    crs="EPSG:32641"  # UTM Zone 41N
)

# Reproject points to WGS84 (EPSG:4326)
gdf_wgs84 = gdf.to_crs("EPSG:4326")

# Directory containing the raster files
raster_dir = Path.cwd()

# List of raster files
raster_files = [
    'LULC_2017_10m.tif', 'LULC_2018_10m.tif', 'LULC_2019_10m.tif',
    'LULC_2021_10m.tif', 'LULC_2022_10m.tif', 'LULC_2023_10m.tif', 'LULC_2024_10m.tif'
]

# Function to extract raster values at point locations with nearest neighbor
def extract_raster_values(raster_path, points_gdf):
    with rasterio.open(raster_path) as src:
        # Verify raster CRS is WGS84
        wgs84_crs = CRS.from_epsg(4326)
        if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:
            raise ValueError(f"Raster {raster_path} is not in WGS84 (got {src.crs})")
        
        # Get point coordinates
        coords = [(x, y) for x, y in zip(points_gdf.geometry.x, points_gdf.geometry.y)]
        
        # Transform coordinates to raster's pixel space
        rows, cols = [], []
        for x, y in coords:
            col, row = src.index(x, y)  # Convert lon/lat to row/col
            rows.append(row)
            cols.append(col)
        
        # Read the raster data
        data = src.read(1)  # Read first (and only) band
        
        # Check if points are within raster bounds
        height, width = data.shape
        values = []
        for row, col in zip(rows, cols):
            if 0 <= row < height and 0 <= col < width:
                values.append(data[row, col])
            else:
                values.append(None)  # Point is outside raster
                print(f"Point at row {row}, col {col} is outside raster bounds")
        
    return values

# Loop through each raster and add values to the GeoDataFrame
for raster_file in raster_files:
    raster_path = raster_dir / raster_file
    if not raster_path.exists():
        print(f"Warning: {raster_path} not found, skipping...")
        continue
    
    year = raster_file.split('_')[1].split('_')[0]  # Extract year
    column_name = f'LULC_{year}'
    
    # Extract values
    gdf[column_name] = extract_raster_values(raster_path, gdf_wgs84)

# Convert back to a regular DataFrame (drop geometry)
result_df = pd.DataFrame(gdf.drop(columns='geometry'))

# Save or print the resulting DataFrame
print(result_df)
result_df.to_csv('output_with_lulc.csv', index=False)

Point at row 23575, col 1145 is outside raster bounds
Point at row 23571, col 1143 is outside raster bounds
Point at row 23567, col 1142 is outside raster bounds
Point at row 23563, col 1141 is outside raster bounds
Point at row 23560, col 1140 is outside raster bounds
Point at row 23556, col 1138 is outside raster bounds
Point at row 23552, col 1137 is outside raster bounds
Point at row 23548, col 1136 is outside raster bounds
Point at row 23544, col 1134 is outside raster bounds
Point at row 23540, col 1133 is outside raster bounds
Point at row 23536, col 1132 is outside raster bounds
Point at row 23533, col 1130 is outside raster bounds
Point at row 23529, col 1129 is outside raster bounds
Point at row 23526, col 1127 is outside raster bounds
Point at row 23523, col 1124 is outside raster bounds
Point at row 23520, col 1122 is outside raster bounds
Point at row 23518, col 1119 is outside raster bounds
Point at row 23516, col 1116 is outside raster bounds
Point at row 23515, col 1112

/tmp/ipykernel_115593/1628730158.py:36: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:
/tmp/ipykernel_115593/1628730158.py:36: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:
/tmp/ipykernel_115593/1628730158.py:36: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:
/tmp/ipykernel_115593/1628730158.py:36: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or not src.crs == wgs84_crs:
/tmp/ipykernel_115593/1628730158.py:36: RasterioDeprecationWarning: is_valid is not useful and will be removed in 2.0.0.
  if not src.crs.is_valid or not src.crs.is_geographic or n